In [ ]:
# !!!!!!
# Attention: Please double check that the code you are reading should be located 
# at '/home/xilinx/jupyter_notebooks/dac_2019_contest/XJTU_Tripler'.
# !!!!!

# ==========================================================================
# Cell 1： Import all libraries and initialize python
# ==========================================================================

import asyncio
from pynq import Overlay
from pynq import DefaultIP
from pynq import Interrupt
from pynq import Xlnk
from pynq.lib import Switch
import numpy as np
from pynq import MMIO
import re
import numpy as np
import sys
import math
import os
import time
from PIL import Image
from matplotlib import pyplot
import cv2
from datetime import datetime
import pynq
from pynq import Xlnk
from pynq import Overlay

from preprocessing import Agent
from preprocessing import BATCH_SIZE
from preprocessing import get_image_path
from IPython.display import display

# import customized lib
import HpuLib

team = 'XJTU_Tripler'
agent = Agent(team)

contest_dir = '/home/xilinx/jupyter_notebooks/dac_2019_contest'
base_dir = contest_dir + '/' + team


In [ ]:
# ==========================================================================
# Cell 2: Download Overlay and initialize PYNQ environments
# ==========================================================================

# Reset PYNQ board
xlnk = Xlnk() 
xlnk.xlnk_reset()

# Download the Overlay
overlay = Overlay(base_dir + '/t.bit')
ps = overlay.bram_ctrl


In [ ]:
# ==========================================================================
# Cell 3: Configure HPU core, and process all images
# ==========================================================================

# ----------------------
# Initialize HPU core
# ----------------------

# Allocate shared memory space from DDR
base_ddr_space = xlnk.cma_array(shape=(1024*1024*12,), dtype=np.int8)

# Initialize the HPU core
hpu = HpuLib.HpuLib(ps, base_ddr_space)

# Download weight parameters
hpu.load_weight(base_dir + '/total_weight.txt')

# Download bias parameters
hpu.load_bias(base_dir + '/total_bias.txt')

# Download the data section of RISC-V
hpu.load_riscv_data_sect(base_dir + '/1_dtcm.verilog')

# Download the code section of RISC-V
hpu.load_riscv_code_sect(base_dir + '/1_itcm.verilog')   

# Start HPU core
hpu.start_hpu()

# ----------------------
# Start to process detection on all images
# ----------------------

# Timing and Energy measuring
interval_time = 0
total_time = 0
total_energy = 0
total_num_img = len(agent.img_list)
result = list()
agent.reset_batch_count()
rails = pynq.get_rails()
print('--------------------------------------------------------------------------------')
print('Processing detection task on all %d images.' % total_num_img)
print('[INFO] This phase may remains about %f mins.' % (total_num_img/2220.))

for i in range(math.ceil(total_num_img/BATCH_SIZE)):
    # get a batch from agent
    batch = agent.send(interval_time, agent.img_batch)

    # choose a single image from the batch
    #first_image = sorted(batch)[0]
    image_batch = batch
    first_image = image_batch[0]

    # timer start when PS reading image
    start = time.time()

    recorder = pynq.DataRecorder(rails['power1'].power)

    with recorder.record(0.01):
        b_size = len(image_batch)

        # read first image
        hpu.shufflenet_v2_read_image(get_image_path(first_image))

        # loop
        for j in range(b_size):
            hpu.shufflenet_v2_start_conv()
            if j > 0:
                result.append(hpu.shufflenet_v2_last_layer_in_c())
            if j < b_size-1:
                hpu.shufflenet_v2_read_image( get_image_path(image_batch[j+1]) )
            # waiting for the convolution finishes
            hpu.shufflenet_v2_wait()

        # calculate last layer of last image
        result.append(hpu.shufflenet_v2_last_layer_in_c())

    # timer stop after PS has received image
    end = time.time()
    t = end - start
    energy = recorder.frame["power1_power"].mean() * t

    # record time consumption
    total_time += t
    # record energy consumption
    total_energy += energy
    
    print('>', end='') if i != 0 else print('[>', end='')

print(']')
print('All images are processed.')


In [ ]:
# ==========================================================================
# Cell 4: Write results to XML
# ==========================================================================

result_rectangle =  result
agent.save_results_xml(result_rectangle, total_time, total_energy)
print("XML results written successfully.")  


In [ ]:
# ==========================================================================
# Cell 5: Clean up
# ==========================================================================
xlnk.xlnk_reset()
print('Clean up the PYNQ environment.')
